## Detroit Blight Ticket Compliance

This project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this project, our task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/).
___
There are two data files for use in training and validating models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Predictions will be given as the probability that the corresponding blight ticket will be paid on time.

We are creating a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier


def blight_model():
    train_data = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_data = pd.read_csv('test.csv')
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    #Merging addresses and latlons
    addresses = addresses.set_index('address').join(latlons.set_index('address'),how = 'left')
    
    #adding adddresses and latlons to the train and test data
    train_data = train_data.set_index('ticket_id').join(addresses.set_index('ticket_id'),how='left')
    test_data = test_data.set_index('ticket_id').join(addresses.set_index('ticket_id'),how='left')
    
    #Removing columns not in test data
    train_remove = [
            'balance_due',
            'collection_status',
            'compliance_detail',
            'payment_amount',
            'payment_date',
            'payment_status']

    train_data.drop(train_remove, axis=1, inplace=True)
    
    # Removing null valued hearing date rows
    train_data = train_data[~train_data['hearing_date'].isnull()]
    
    # Remove String and Extra data
    string_remove_list = ['violator_name', 'zip_code', 'country', 'city',
            'inspector_name', 'violation_street_number', 'violation_street_name',
            'violation_zip_code', 'violation_description',
            'mailing_address_str_number', 'mailing_address_str_name',
            'non_us_str_code', 'agency_name', 'state', 'disposition',
            'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']

    train_data.drop(string_remove_list, axis=1, inplace=True)
    test_data.drop(string_remove_list, axis=1, inplace=True)
    
    # Fill NA Lat Lon Values by pad/ffill
    train_data.lat.fillna(method='pad', inplace=True)
    train_data.lon.fillna(method='pad', inplace=True)
    test_data.lat.fillna(method='pad', inplace=True)
    test_data.lon.fillna(method='pad', inplace=True)
    
    #Removing data with compliance equal to Nan
    train_data = train_data[(train_data['compliance']==0) | (train_data['compliance']==1)]
    
    #Creating X_train,y_train and X_test
    X_train = train_data.drop('compliance',axis=1)
    y_train = train_data.compliance
    X_test = test_data
    
    #Creating the Classifier(Params as per gridsearch)
    clf = RandomForestClassifier(n_estimators=11 , max_depth= 5)

    #Training the classifier
    clf.fit(X_train,y_train)
    
    #Predicting probabilities of paying the fine
    y_proba = clf.predict_proba(X_test)[:,1]
    
    #Adding probabilities to the data
    X_test['compliance'] = y_proba
    ans = X_test.compliance
    
    return ans

In [2]:
blight_model()

C:\Users\omsan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


ticket_id
284932    0.063966
285362    0.025126
285361    0.060086
285338    0.074056
285346    0.077592
            ...   
376496    0.024577
376497    0.024577
376499    0.065329
376500    0.065329
369851    0.296290
Name: compliance, Length: 61001, dtype: float64

In [3]:
#Getting Best Params using gridsearch

#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV

#a,b,c,d = train_test_split(X_train, y_train, random_state = 0)
    
#clf = RandomForestClassifier()   
#grid_values = {'n_estimators': [9, 10, 11], 'max_depth': [1,2,3,4,5] }  
#grid_clf = GridSearchCV(clf, param_grid = grid_values)
#grid_clf.fit(a,c)

#grid_clf.best_params_

#As per the results 

#clf = RandomForestClassifier(n_estimators=11 , max_depth= 5)
#clf.score(b,d)